In [17]:
import sqlalchemy
from sqlalchemy import create_engine, text
from tabulate import tabulate

In [3]:
DB_USER = "myuser"
DB_PASS = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydb"

In [4]:
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

In [5]:
engine = create_engine(DATABASE_URL)

In [6]:
# Function to execute SQL statements safely
def execute_sql(sql_statement):
    try:
        with engine.connect() as connection:
            connection.execute(text(sql_statement))
            connection.commit()
            print(f"SQL statement executed successfully:\n{sql_statement}")
    except sqlalchemy.exc.ProgrammingError as e:
        print(f"Error executing SQL statement:\n{sql_statement}\nError: {e}")
    except Exception as e:
        print(f"An unexpected error occurred:\n{sql_statement}\nError: {e}")

In [31]:
try:
    with engine.connect() as connection:
        connection.execute(text("DROP TABLE Customers CASCADE;"))
        
        connection.commit()

except Exception as e:
    print("Error querying table:", e)

In [32]:
execute_sql("""
    CREATE TABLE Customers (
        CustomerID SERIAL PRIMARY KEY,
        FirstName VARCHAR(255),
        LastName VARCHAR(255)
    );
""")

SQL statement executed successfully:

    CREATE TABLE Customers (
        CustomerID SERIAL PRIMARY KEY,
        FirstName VARCHAR(255),
        LastName VARCHAR(255)
    );



In [43]:
execute_sql("""
    CREATE TABLE Orders (
        OrderID SERIAL PRIMARY KEY,
        CustomerID INT REFERENCES Customers(CustomerID),
        OrderDate DATE
    );
""")

SQL statement executed successfully:

    CREATE TABLE Orders (
        OrderID SERIAL PRIMARY KEY,
        CustomerID INT REFERENCES Customers(CustomerID),
        OrderDate DATE
    );



In [ ]:
execute_sql("""
    CREATE TABLE IF NOT EXISTS Employees (
        EmployeeID SERIAL PRIMARY KEY,
        EmployeeName VARCHAR(255),
        ManagerID INT REFERENCES Employees(EmployeeID) NULL
    );
""")

SQL statement executed successfully:

    CREATE TABLE IF NOT EXISTS Employees (
        EmployeeID SERIAL PRIMARY KEY,
        EmployeeName VARCHAR(255),
        ManagerID INT REFERENCES Employees(EmployeeID) NULL  -- Self-referencing foreign key
    );



## Populate tables

In [33]:
execute_sql("""
    INSERT INTO Customers (FirstName, LastName) VALUES
    ('John', 'Doe'),
    ('Jane', 'Smith'),
    ('Peter', 'Jones');
""")

SQL statement executed successfully:

    INSERT INTO Customers (FirstName, LastName) VALUES
    ('John', 'Doe'),
    ('Jane', 'Smith'),
    ('Peter', 'Jones');



In [38]:
try:
    with engine.connect() as connection:
     
        result = connection.execute(text("SELECT * FROM Customers"))

        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)

except Exception as e:
    print("Error querying table:", e)

+--------------+-------------+------------+
|   customerid | firstname   | lastname   |
+==============+=============+============+
|            1 | John        | Doe        |
+--------------+-------------+------------+
|            2 | Jane        | Smith      |
+--------------+-------------+------------+
|            3 | Peter       | Jones      |
+--------------+-------------+------------+


In [44]:
execute_sql("""
    INSERT INTO Orders (CustomerID, OrderDate) VALUES
    (1, '2023-11-20'),
    (1, '2023-11-21'),
    (2, '2023-11-22');
""")

SQL statement executed successfully:

    INSERT INTO Orders (CustomerID, OrderDate) VALUES
    (1, '2023-11-20'),
    (1, '2023-11-21'),
    (2, '2023-11-22');



In [50]:
try:
    with engine.connect() as connection:
     
        result = connection.execute(text("SELECT * FROM Orders"))

        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)

except Exception as e:
    print("Error querying table:", e)

+-----------+--------------+-------------+
|   orderid |   customerid | orderdate   |
+===========+==============+=============+
|         1 |            1 | 2023-11-20  |
+-----------+--------------+-------------+
|         2 |            1 | 2023-11-21  |
+-----------+--------------+-------------+
|         3 |            2 | 2023-11-22  |
+-----------+--------------+-------------+


In [ ]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("""
            SELECT * FROM GetCustomerByID(:custID)
        """), {"custID": 1})

        rows = result.fetchall()
        column_names = result.keys()
        table = tabulate(rows, headers=column_names, tablefmt="grid")
        print(table)

except Exception as e:
    print("Error using function:", e)

+--------------+-------------+------------+----------------------+
|   customerid | firstname   | lastname   | email                |
+==============+=============+============+======================+
|            1 | John        | Doe        | john.doe@example.com |
+--------------+-------------+------------+----------------------+


In [12]:
execute_sql("""
    INSERT INTO Employees (EmployeeName, ManagerID) VALUES
    ('Alice', NULL),        -- Alice is the top-level manager
    ('Bob', 1),            -- Bob reports to Alice
    ('Charlie', 1),        -- Charlie reports to Alice
    ('David', 2);          -- David reports to Bob
""")

SQL statement executed successfully:

    INSERT INTO Employees (EmployeeName, ManagerID) VALUES
    ('Alice', NULL),        -- Alice is the top-level manager
    ('Bob', 1),            -- Bob reports to Alice
    ('Charlie', 1),        -- Charlie reports to Alice
    ('David', 2);          -- David reports to Bob




### INNER JOIN

*   **Definition:** Returns rows only when there is a match in *both* tables based on the specified join condition.
*   **Behavior:** Rows without a matching value in the other table are excluded from the result set.
*   **Example:**

    ```sql
    SELECT Orders.OrderID, Customers.FirstName, Customers.LastName
    FROM Orders
    INNER JOIN Customers ON Orders.CustomerID = Customers.CustomerID;
    ```

In [46]:
try:
    with engine.connect() as connection:
        
        result = connection.execute(text(
                """
                    SELECT Orders.OrderID, Customers.FirstName, Customers.LastName
                    FROM Orders
                    INNER JOIN Customers ON Orders.CustomerID = Customers.CustomerID;  
                """
        ))

        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)

except Exception as e:
    print("Error querying table:", e)

+-----------+-------------+------------+
|   orderid | firstname   | lastname   |
+===========+=============+============+
|         1 | John        | Doe        |
+-----------+-------------+------------+
|         2 | John        | Doe        |
+-----------+-------------+------------+
|         3 | Jane        | Smith      |
+-----------+-------------+------------+


### LEFT JOIN (or LEFT OUTER JOIN)

*   **Definition:** Returns all rows from the *left* table and matching rows from the *right* table.
*   **Behavior:** If there is no match in the right table for a row in the left table, the right table's columns will have `NULL` values in the result set.  All rows from the left table will be included regardless of whether there's a match in the right table.
*   **Example:**

    ```sql
    SELECT Customers.FirstName, Customers.LastName, Orders.OrderID
    FROM Customers
    LEFT JOIN Orders ON Customers.CustomerID = Orders.CustomerID;
    ```

In [ ]:
try:
    with engine.connect() as connection:
        
        result = connection.execute(text(
                """
                    SELECT Customers.FirstName, Customers.LastName, Orders.OrderID
                    FROM Customers
                    LEFT JOIN Orders ON Customers.CustomerID = Orders.CustomerID; 
                """
        ))

        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)

except Exception as e:
    print("Error querying table:", e)

+-------------+------------+-----------+
| firstname   | lastname   |   orderid |
+=============+============+===========+
| John        | Doe        |         1 |
+-------------+------------+-----------+
| John        | Doe        |         2 |
+-------------+------------+-----------+
| Jane        | Smith      |         3 |
+-------------+------------+-----------+
| Peter       | Jones      |           |
+-------------+------------+-----------+


### RIGHT JOIN (or RIGHT OUTER JOIN)

*   **Definition:** Returns all rows from the *right* table and matching rows from the *left* table.
*   **Behavior:** If there is no match in the left table for a row in the right table, the left table's columns will have `NULL` values in the result set.  All rows from the right table will be included regardless of whether there's a match in the left table.
*   **Example:**

    ```sql
    SELECT Customers.FirstName, Customers.LastName, Orders.OrderID
    FROM Customers
    RIGHT JOIN Orders ON Customers.CustomerID = Orders.CustomerID;
    ```


In [53]:
try:
    with engine.connect() as connection:
        
        result = connection.execute(text(
                """
                    SELECT Customers.FirstName, Customers.LastName, Orders.OrderID
                    FROM Customers
                    RIGHT JOIN Orders ON Orders.CustomerID = Customers.CustomerID;
                """
        ))

        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)

except Exception as e:
    print("Error querying table:", e)

+-------------+------------+-----------+
| firstname   | lastname   |   orderid |
+=============+============+===========+
| John        | Doe        |         1 |
+-------------+------------+-----------+
| John        | Doe        |         2 |
+-------------+------------+-----------+
| Jane        | Smith      |         3 |
+-------------+------------+-----------+


### FULL OUTER JOIN

*   **Definition:** Returns all rows when there is a match in *one* of the tables.  Combines the results of both LEFT and RIGHT joins.
*   **Behavior:** If there is no match in the left table for a row in the right table, the left table's columns will have `NULL` values. If there is no match in the right table for a row in the left table, the right table's columns will have `NULL` values.  Includes all rows from both tables.
*   **Example:**

    ```sql
    SELECT Customers.FirstName, Customers.LastName, Orders.OrderID
    FROM Customers
    FULL OUTER JOIN Orders ON Customers.CustomerID = Orders.CustomerID;
    ```


In [54]:
try:
    with engine.connect() as connection:
        
        result = connection.execute(text(
                """
                    SELECT Customers.FirstName, Customers.LastName, Orders.OrderID
                    FROM Customers
                    FULL OUTER JOIN Orders ON Customers.CustomerID = Orders.CustomerID;
                """
        ))

        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)

except Exception as e:
    print("Error querying table:", e)

+-------------+------------+-----------+
| firstname   | lastname   |   orderid |
+=============+============+===========+
| John        | Doe        |         1 |
+-------------+------------+-----------+
| John        | Doe        |         2 |
+-------------+------------+-----------+
| Jane        | Smith      |         3 |
+-------------+------------+-----------+
| Peter       | Jones      |           |
+-------------+------------+-----------+


### SELF JOIN

*   **Definition:** Joins a table to itself. Used when there's a relationship between rows *within* the same table (often used for hierarchical data, like organizational charts or parent-child relationships).
*   **Behavior:**  Requires using aliases for the table name to distinguish between the different instances of the table being joined.
*   **Example:**

    ```sql
    SELECT e1.EmployeeName, e2.EmployeeName AS ManagerName
    FROM Employees e1
    LEFT JOIN Employees e2 ON e1.ManagerID = e2.EmployeeID;
    ```

    *   `e1`: Alias for the `Employees` table, representing an employee.
    *   `e2`: Alias for the `Employees` table, representing the manager.
    *   `e1.ManagerID = e2.EmployeeID`: The join condition links an employee's `ManagerID` to another employee's `EmployeeID`, establishing the manager-employee relationship.  A `LEFT JOIN` is used to include employees who don't have a manager (top-level employees).

In [55]:
try:
    with engine.connect() as connection:
        
        result = connection.execute(text(
                """
                    SELECT e1.EmployeeName, e2.EmployeeName AS ManagerName
                    FROM Employees e1
                    LEFT JOIN Employees e2 ON e1.ManagerID = e2.EmployeeID;
                """
        ))

        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)

except Exception as e:
    print("Error querying table:", e)

+----------------+---------------+
| employeename   | managername   |
+================+===============+
| Alice          |               |
+----------------+---------------+
| Bob            | Alice         |
+----------------+---------------+
| Charlie        | Alice         |
+----------------+---------------+
| David          | Bob           |
+----------------+---------------+
